In [1]:
import json
import time 
from kafka import KafkaProducer

In [2]:
!pip show kafka

Name: kafka
Version: 1.3.5
Summary: Pure Python client for Apache Kafka
Home-page: https://github.com/dpkp/kafka-python
Author: Dana Powers
Author-email: dana.powers@gmail.com
License: Apache License 2.0
Location: /home/ramazan/DE_zoomcamp_2024/module_6/.venv/lib/python3.9/site-packages
Requires: 
Required-by: 


In [3]:
!pip show kafka-python

Name: kafka-python
Version: 1.4.7
Summary: Pure Python client for Apache Kafka
Home-page: https://github.com/dpkp/kafka-python
Author: Dana Powers
Author-email: dana.powers@gmail.com
License: Apache License 2.0
Location: /home/ramazan/DE_zoomcamp_2024/module_6/.venv/lib/python3.9/site-packages
Requires: 
Required-by: robinhood-aiokafka


In [4]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'


topic_name = 'test-topic'

In [5]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [6]:
producer.bootstrap_connected()

True

In [7]:
t0 = time.time()

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)
    
t1 = time.time()
print(f'sending messages took {(t1 - t0):.2f} seconds')

t0 = time.time()
producer.flush()
t1 = time.time()
print(f'flushing took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
sending messages took 0.51 seconds
flushing took 0.00 seconds


In [8]:
import pandas as pd

In [9]:
url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz'
df = pd.read_csv(url)

/tmp/ipykernel_586795/518789981.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [10]:
df_green = df[['lpep_pickup_datetime',
'lpep_dropoff_datetime',
'PULocationID',
'DOLocationID',
'passenger_count',
'trip_distance',
'tip_amount']]

In [11]:
df_green.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [12]:
green_topic = "green-trips-2"

t0 = time.time()
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(green_topic, value=row_dict)
t1 = time.time()

producer.flush()

print(f"DONE in {t1-t0} seconds")

DONE in 87.5462577342987 seconds


In [13]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/ramazan/spark/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ramazan/.ivy2/cache
The jars for the packages stored in: /home/ramazan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9d159c39-5aad-4e88-b03c-7193553b31c9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: re

24/03/15 12:01:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips-2") \
    .option("startingOffsets", "earliest") \
    .load()

In [15]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/15 12:01:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bcaee210-baa2-449e-9087-08e6c2b4e359. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 12:01:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Row(lpep_pickup_datetime='2019-10-01 00:57:44', lpep_dropoff_datetime='2019-10-01 01:06:59', PULocationID=25, DOLocationID=228, passenger_count=1.0, trip_distance=2.62, tip_amount=0.0)


In [16]:
query.stop()

In [17]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [18]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [19]:
query_2 = green_stream.writeStream.foreachBatch(peek).start()

24/03/15 12:01:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f9a6095e-d907-491a-a120-5a71fe724808. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 12:01:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [20]:
from pyspark.sql.functions import current_timestamp, window, col, count

In [21]:
green_stream = green_stream.withColumn("timestamp", current_timestamp())

In [22]:
from pyspark.sql.functions import window, col, count

popular_destinations = (green_stream
          .groupBy(window(col("timestamp"), "5 minutes"), "DOLocationID")
          .agg(count("*").alias("count"))
          .orderBy(col("count").desc())
         )

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/15 12:01:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c17ba431-f647-43ae-bf89-cb597ffcfbec. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/15 12:01:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|74          |35478|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|42          |31883|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|41          |28121|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|75          |25677|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|129         |23859|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|7           |23062|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|166         |21688|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|236         |15826|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|223         |15083|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|238         |14636|
|{2024-03-15 12:00:00, 2024-03-15 12:05:00}|82          |14581|
|{2024-